# Exercise 10: Psycopg − programmatic access to Postgres

This exercise focuses on using a SQL database, Postgres here, "programmatically": this means we will be writing some Python code to interact with the database, similarly to how a developer may build a web application, such as an online store. We will also use transactions to make sure our updates to the database in response to the user's request are atomic and consistent.

First off, let's set up a simple database representing the product offering and the orders received for a fictional online store selling kitchen utensils.

In [1]:
%load_ext sql

Connect to the database. This should work for the docker-based setup. You should change the connection string if you have installed postgres manually.

In [2]:
%sql postgresql://postgres:example@db:5432

In [3]:
%%sql
drop schema if exists shop cascade;

create schema shop;

create table shop.products(
    id serial primary key,
    description varchar not null,
    price numeric(7,2) not null,
    in_stock integer not null);

create table shop.orders(
    id serial primary key,
    time timestamp,
    product_id integer,
    quantity integer,
    constraint product_id
        foreign key (product_id) references shop.products (id)
        match simple on update restrict on delete restrict);

insert into shop.products (
    id, description, price, in_stock) values
    ( 1, 'Bread knife', 4.07, 8 ),
    ( 2, 'Cheesecloth', 7.32, 3 ),
    ( 3, 'Cleaver',     12.23,7 ),
    ( 4, 'Colander',    3.00, 2 ),
    ( 5, 'Grater',      7.99, 13),
    ( 6, 'Ladle',       4.50, 23),
    ( 7, 'Mandoline',   18.12,11),
    ( 8, 'Oven mitt',   9.99, 6 ),
    ( 9, 'Rolling pin', 4.85, 27),
    (10, 'Sieve',       6.89, 16),
    (11, 'Tin opener',  7.00, 2 );

 * postgresql://postgres:***@db:5432
Done.
Done.
Done.
Done.
11 rows affected.


[]

The product catalog also tracks how many items are in stock (the `in_stock` column), and each order can only consist of a single product, which is possibly unrealistic but simplifies some of the code in this exercise.

Note that we're using `serial` for the type of the `id`s: this is a special `integer` type that instructs postgres to generate a fresh id whenever we insert a new row without explicitly specifying an id.

## Psycopg2
Psycopg2 is (one of) the python libraries to interact with a Postgres instance. Let's import it.
We're also importing `datetime`, which we'll need later to construct a `datetime` to insert as the order time.
You can find the documentation for psycopg2 here: [https://www.psycopg.org/docs/](https://www.psycopg.org/docs/)

In [4]:
import psycopg2
import datetime

We can ask `psycopg2` to connect to the database and give us back a handle to the connection as follows. This should work for the docker-based setup. You should change the connection string if you have installed postgres manually.

In [5]:
conn = psycopg2.connect('postgresql://postgres:example@db:5432')
conn

<connection object at 0xffff93ff9e00; dsn: 'user=postgres password=xxx host=db port=5432', closed: 0>

By the way, this is the same thing that happens under the hood when you use the special `%sql` cell in `jupyter`.

*NOTE:* if you start getting unexpected errors when running queries, it may be that the `cursor` or connection are in an invalid state; if that happens you can try to call

In [6]:
conn.rollback()

an make a new cursor. This often resolves this problem. If it doesn't, you can try and re-start the python kernel (in the menu: "Kernel" -> "Restart") but then you need to re-run all cells.

## Executing queries − Product list

We'll start with using python and `psycopg2` to run a query against the database to obtain a product list.

Here's an example of how you can write such a query, your task will then be to adapt this code to write a function that lists products in the shop.

First of all, we need to create a new `cursor`, which we'll use to perform database operations. As the documentation says,
> The class `cursor` allows interaction with the database:
> * send commands to the database using methods such as `execute()` and `executemany()`,
> * retrieve data from the database by iteration or using methods such as `fetchone()`, `fetchmany()`, `fetchall()`.

In [7]:
cur = conn.cursor()

Then we can start executing a sql query, with `execute`. In the following example, we're listing the product ids and their descriptions:

In [8]:
cur.execute('select id, description from shop.products')

As you can see, this only sends the query to the database, but doesn't start obtaining the results yet, to do that we have a few different options, but one of the easiest is to iterate over the cursor:

In [9]:
for record in cur:
    print(record)

(1, 'Bread knife')
(2, 'Cheesecloth')
(3, 'Cleaver')
(4, 'Colander')
(5, 'Grater')
(6, 'Ladle')
(7, 'Mandoline')
(8, 'Oven mitt')
(9, 'Rolling pin')
(10, 'Sieve')
(11, 'Tin opener')


We get one tuple per record (row), with one element per field.

We can also use `fetchall` to obtain all results in a single list. However, because we've already iterated over the cursor, we need to re-run the query first, otherwise we'd just get an empty resultset.

In [10]:
cur = conn.cursor()
cur.execute('select id, description from shop.products')
cur.fetchall()

[(1, 'Bread knife'),
 (2, 'Cheesecloth'),
 (3, 'Cleaver'),
 (4, 'Colander'),
 (5, 'Grater'),
 (6, 'Ladle'),
 (7, 'Mandoline'),
 (8, 'Oven mitt'),
 (9, 'Rolling pin'),
 (10, 'Sieve'),
 (11, 'Tin opener')]

We can also use string formatting to display this in a more human readable manner, for example:

In [11]:
cur = conn.cursor()
cur.execute('select id, description from shop.products')
for record in cur:
    print("{}: {}".format(*record))

1: Bread knife
2: Cheesecloth
3: Cleaver
4: Colander
5: Grater
6: Ladle
7: Mandoline
8: Oven mitt
9: Rolling pin
10: Sieve
11: Tin opener


We should also remember to close the cursors once we're done with them, to release resources:

In [12]:
cur.close()

Now that we have discussed how to use a cursor, your task is to write the function `show_products`, that when called prints all products in the database, displaying their description, price, and the number of items in stock. Here's an example of the output you should obtain:

    Bread knife, 4.07 CHF (8 available)
    Cheesecloth, 7.32 CHF (3 available)
    Cleaver, 12.23 CHF (7 available)
    Colander, 3.00 CHF (2 available)
    Grater, 7.99 CHF (13 available)
    Ladle, 4.50 CHF (23 available)
    Mandoline, 18.12 CHF (11 available)
    Oven mitt, 9.99 CHF (6 available)
    Rolling pin, 4.85 CHF (27 available)
    Sieve, 6.89 CHF (16 available)
    Tin opener, 7.00 CHF (2 available)

In [13]:
def show_products():
    cur = conn.cursor()
    cur.execute('select description, price, in_stock from shop.products')
    for record in cur:
        name, price, number = record
        print(f"{name}, {price} CHF, ({number} available)")

Run it and check it returns the correct results:

In [14]:
show_products()

Bread knife, 4.07 CHF, (8 available)
Cheesecloth, 7.32 CHF, (3 available)
Cleaver, 12.23 CHF, (7 available)
Colander, 3.00 CHF, (2 available)
Grater, 7.99 CHF, (13 available)
Ladle, 4.50 CHF, (23 available)
Mandoline, 18.12 CHF, (11 available)
Oven mitt, 9.99 CHF, (6 available)
Rolling pin, 4.85 CHF, (27 available)
Sieve, 6.89 CHF, (16 available)
Tin opener, 7.00 CHF, (2 available)


Great, now our customer knows what products are available. In a real web store, we would send this information as part of a web page to the user.

## Prepared statements − Finding products below a certain price

This next section explains how we can pass in a parameter to the query that the user has provided. In the lecture, you've seen how you can use prepared statements for this: let's see how to do it with `psycopg2`.

Let's start with a simple query: displaying the product description for a certain product id. We can use `%s` as a placeholder in the sql for the query parameters. Then we can use the second optional parameter to `execute` to pass the value for that placeholder, as follows.

In [15]:
cur = conn.cursor()
product_id = 3
cur.execute('select description from shop.products where id = %s;', (product_id,))
print(cur.fetchall())

[('Cleaver',)]


Now that we have the ability to parametrize queries, your task is to write a query that helpfully returns to the customer all the products within a certain price range.
Because the focus of this exercise is on accessing a database programmatically and not on how to write queries, here's the query you'd write to obtain all products between 4CHF and 5CHF:

    select description, price, in_stock from shop.products where price >= 4 and price <= 5
    
Let's see what happens when we run it:

In [16]:
cur = conn.cursor()
product_id = 3
cur.execute('select description, price, in_stock from shop.products where price >= 4 and price <= 5')
print(cur.fetchall())

[('Bread knife', Decimal('4.07'), 8), ('Ladle', Decimal('4.50'), 23), ('Rolling pin', Decimal('4.85'), 27)]


Your task is now to write a function `get_products_within_price_range` that takes the lower and upper bound for the price and prints the products within that price range. You can use the same formatting as `show_products`.
When called with `get_products_within_price_range(5, 6)` it should print:

    Cheesecloth, 7.32 CHF (3 available)
    Grater, 7.99 CHF (13 available)
    Sieve, 6.89 CHF (16 available)
    Tin opener, 7.00 CHF (2 available)

In [17]:
def get_products_within_price_range(price_lower_bound, price_upper_bound):
    cur = conn.cursor()
    cur.execute(f'select description, price, in_stock from shop.products where price >= {price_lower_bound} and price <= {price_upper_bound}')
    for record in cur:
        name, price, number = record
        print(f"{name}, {price} CHF, ({number} available)")

Run it and check it returns the correct results:

In [18]:
get_products_within_price_range(6, 8)

Cheesecloth, 7.32 CHF, (3 available)
Grater, 7.99 CHF, (13 available)
Sieve, 6.89 CHF, (16 available)
Tin opener, 7.00 CHF, (2 available)


We can also use the results from `cursor` to extract data from the database and use it programmatically. The following function returns the product id given a certain product description:

In [19]:
def get_product_id_from_description(product_description):
    cur = conn.cursor()
    cur.execute('select id from shop.products where description = %s;', (product_description,))
    results = cur.fetchall()
    assert(len(results) == 1)
    return results[0][0]

In [20]:
get_product_id_from_description('Mandoline')

7

## Updates from `python` − Placing an order

We will now take a look at how to update the database using cursors and prepared statemens via `psycopg2`.

As an example, imagine the shop has decided to change the price oven mitts to 8.99CHF. They could write a query as follows:

In [21]:
cur = conn.cursor()
cur.execute('update shop.products set price = 8.99 where id = %s', (get_product_id_from_description('Oven mitt'),))
#cur.execute('update shop.products set in_stock = -1 where id = %s', (get_product_id_from_description('Oven mitt'),))
conn.commit()

We need to call `commit` because running a query automatically starts a transaction for a `connection` in `psycopg2`, and we need to commit the transaction so that the changes become visible globally.

Let's check that oven mitts are now within the right price range:

In [22]:
get_products_within_price_range(8.5, 9.5)

Oven mitt, 8.99 CHF, (6 available)


This should show

    Oven mitt, 8.99 CHF (6 available)

Cool! Your task is to write a function `place_order` that takes a product id and a quantity and updates the `orders` table to add a new order for that product and quantity. To start, don't worry about updating the inventory (the `in_stock` field in `products`).

In [23]:
import datetime

In [24]:
def place_order(product_id, quantity):
    now = datetime.datetime.now()
    t = now.strftime('%Y-%m-%d %H:%M:%S') 
    cur = conn.cursor()
    cur.execute(f"INSERT INTO shop.orders (time, product_id, quantity) VALUES ('{t}',{product_id},{quantity})")
    conn.commit()

Let's place an order for 2 mandolines!

In [25]:
place_order(get_product_id_from_description('Mandoline'), 2)

Now let's write a function to show the orders. We are going to need our first join here to show the product description for the orders. Your task is to write `show_orders`, which prints each order id, time, the related product description, and the order quantity.

The output should look like this:

    order 1: 2x Mandoline (on 2020-11-25 14:20:19.007563)

In [26]:
%%sql
SELECT shop.orders.id, shop.orders.quantity, shop.products.description, shop.orders.time
FROM shop.products 
INNER JOIN shop.orders ON shop.products.id=shop.orders.product_id; 

 * postgresql://postgres:***@db:5432
1 rows affected.


id,quantity,description,time
1,2,Mandoline,2022-11-29 21:43:37


In [27]:
def show_orders():
    cur = conn.cursor()
    cur.execute("SELECT shop.orders.id, shop.orders.quantity, shop.products.description, shop.orders.time FROM shop.products INNER JOIN shop.orders ON shop.products.id=shop.orders.product_id")
    for record in cur:
        idd, q, desc, time = record
        print(f"order {idd}: {q}x {desc} (on {time})")

Let's run it:

In [28]:
show_orders()

order 1: 2x Mandoline (on 2022-11-29 21:43:37)


Ok, this works, but when we place an order we should also first check that there's sufficient inventory. We should also add the order and update the related product's inventory (`in_stock`) in the same transaction. This is necessary to ensure we can't concurrently accept two orders that we are not able to fulfill because we don't have sufficient inventory.

Your task is to complete `place_order_and_update_stock` to perform the transaction. As previously mentioned, running a query starts a new transaction, which continues till we call `commit` or `rollback` on the connection.
We're wrapping the whole body in a `try` block so that, in case there is an error (e.g. a constraint violation in the database query) we can rollback the whole transaction (with `conn.rollback()` in the `except` block).

The function should return `True` if the order was placed successfully, and `False` otherwise. The function must fail if there is insufficient inventory and must never update `in_stock` to a negative value.

In [29]:
conn.rollback()

In [30]:
def place_order_and_update_stock(product_id, quantity):
    cur = conn.cursor()
    time = datetime.datetime.now()
    cur.execute('select in_stock from shop.products where id = %s', (product_id,))
    results = cur.fetchall()
    how_many_in_stock = results[0][0]
    if quantity <= how_many_in_stock:
        how_many_after = how_many_in_stock - quantity
        cur.execute(f'update shop.products set in_stock = {how_many_after} where id = {product_id}')
        place_order(product_id, quantity)
        conn.commit()
        return True
    else:
        return False

Ok, let's pick a product:

In [31]:
show_products()

Bread knife, 4.07 CHF, (8 available)
Cheesecloth, 7.32 CHF, (3 available)
Cleaver, 12.23 CHF, (7 available)
Colander, 3.00 CHF, (2 available)
Grater, 7.99 CHF, (13 available)
Ladle, 4.50 CHF, (23 available)
Mandoline, 18.12 CHF, (11 available)
Rolling pin, 4.85 CHF, (27 available)
Sieve, 6.89 CHF, (16 available)
Tin opener, 7.00 CHF, (2 available)
Oven mitt, 8.99 CHF, (6 available)


Let's try and order 10 tin openers:

In [32]:
place_order_and_update_stock(get_product_id_from_description('Tin opener'), 10)

False

This of course should fail (it should return `False`), as we don't have enough tin openers in stock.

Let's try with one, then:

In [33]:
place_order_and_update_stock(get_product_id_from_description('Tin opener'), 1)

True

That should have worked! Let's check the orders:

In [34]:
show_orders()

order 1: 2x Mandoline (on 2022-11-29 21:43:37)
order 2: 1x Tin opener (on 2022-11-29 21:43:50)


This should have returned the two orders we've made so far:

    order 1: 2x Mandoline (on 2020-11-25 14:20:19.007563)
    order 2: 1x Tin opener (on 2020-11-25 14:37:43.065384)

Ok, that's it. This would be a good start to set up an online store of kitchen utensils.